In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import layers, losses
# from tensorflow.keras.models import Model
# import modred as mr
from sklearn.decomposition import PCA, KernelPCA
from scipy import linalg
from sklearn.preprocessing import normalize
from scipy import fftpack

In [66]:
mpl.use('Agg')

density_df_400_2013 = pd.read_csv('Data/2013_HASDM_500-575KM.den', delim_whitespace=True,
                                  header=None)
density_np_400_2013 = pd.DataFrame.to_numpy(density_df_400_2013)
del density_df_400_2013

# density_df_400_2014 = pd.read_csv('Data/2014_HASDM_400-475KM.den', delim_whitespace=True,
#                                   header=None)
# density_np_400_2014 = pd.DataFrame.to_numpy(density_df_400_2014)
# del density_df_400_2014
# density_df_500_2013 = pd.read_csv('Data/2013_HASDM_500-575KM.den', delim_whitespace=True,
#                                   header=None)
# density_np_500_2013 = pd.DataFrame.to_numpy(density_df_500_2013)
# del density_df_500_2013
nt = 19
nphi = 24

t = np.linspace(-np.pi / 2, np.pi / 2, nt)
phi = np.linspace(0, np.deg2rad(345), nphi)

# max_rho1 = np.max(density_np_400_2013[:, 10])
# max_rho2 = np.max(density_np_400_2014[:, 10])
# max_rho = np.max(np.array([max_rho1, max_rho2]))
# density_np_400_2013[:, 10] = density_np_400_2013[:, 10] / max_rho
# density_np_400_2014[:, 10] = density_np_400_2014[:, 10] / max_rho

max_rho = np.max(density_np_400_2013[:, 10])
density_np_400_2013[:, 10] = density_np_400_2013[:, 10] / max_rho

rho_list = []
rho_list1 = []
rho_list2 = []

for i in range(int(1331520 / (nt * nphi))):  # 1335168
    rho_400_i_2013 = density_np_400_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    rho_polar_400_i_2013 = np.reshape(rho_400_i_2013, (nt, nphi, 4))

    # rho_400_i_2014 = density_np_400_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    # rho_polar_400_i_2014 = np.reshape(rho_400_i_2014, (nt, nphi, 4))

    # rho_500_i = density_np_500_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    # rho_polar_500_i = np.reshape(rho_500_i, (nt, nphi, 4))
    #
    # rho_polar = np.concatenate(
    #     (rho_polar_400_i_2013, rho_polar_400_i_2014), axis=2)

    rho_polar_2013 = rho_polar_400_i_2013
    # rho_polar_2014 = rho_polar_400_i_2014

    rho_list1.append(rho_polar_2013)
    # rho_list2.append(rho_polar_2014)


# rho1 = np.array(rho_list1)
# rho2 = np.array(rho_list2)
# rho = np.concatenate((rho1, rho2), axis=0)  # Shape: (5840, 19, 24, 4)

rho = np.array(rho_list1)

rho_zeros = np.zeros((2920, 20, 24, 4))  # 5840, 20, 24, 4
rho_zeros[:, :nt, :nphi, :] = rho
del rho_list1, rho  #, rho_list2, rho1, rho2

training_data = rho_zeros[:2000]
validation_data = rho_zeros[2000:]

training_data_resh = np.reshape(training_data, newshape=(2000, 20*24*4))

In [67]:
def cosine_kernel(x, y):
    x_normalized = normalize(x, copy=True)
    if x is y:
        y_normalized = x_normalized
    else:
        y_normalized = normalize(y, copy=True)
    kernels = np.dot(x_normalized, y_normalized.T)
    return kernels


def my_kernel(X, Y):
    # The best one so far: np.dot(np.sin(X), np.sin(Y).T). It may be better than the linear one if a nonlinear
    # pre-image would exist
    return np.dot(np.sin(X), np.sin(Y).T)


In [68]:
# nPoints_val = 920  # 840
# validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))
# rhoavg = np.mean(validation_data_resh, axis=0)  # Compute mean
# rho_msub_val = validation_data_resh.T - np.tile(rhoavg, (nPoints_val, 1)).T  # Mean-subtracted data

# print(training_data_resh.shape)
rhoavg = np.mean(training_data_resh, axis=0)  # Compute mean
nPoints = 2000
rho_msub = training_data_resh.T - np.tile(rhoavg, (nPoints, 1)).T  # Mean-subtracted data
num_modes = 10

In [69]:
kpca = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=0.11)  #  0.11
kpca_adv = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=0.11,
                     alpha=1e-11)  #  0.11
# kpca1 = KernelPCA(n_components=num_modes, kernel="poly", fit_inverse_transform=True, degree=5, gamma=1.4e-4, coef0=1e-5)
kpca2 = KernelPCA(n_components=num_modes, kernel="sigmoid", fit_inverse_transform=True, gamma=1.4e-2, coef0=1e-5)
kpca4 = KernelPCA(n_components=num_modes, kernel="laplacian", fit_inverse_transform=True, gamma=1e-2)
fftout = fftpack.fftn(rho_msub.T)

# pcam = KernelPCA(n_components=num_modes, kernel="precomputed")  # , fit_inverse_transform=True, gamma=10
pca = PCA(n_components=num_modes)
pca_fft = PCA(n_components=num_modes)

X_pca_lin = pca.fit_transform(rho_msub.T)
X_pca_lin_fft = pca_fft.fit_transform(fftout.real)

# gram = cosine_kernel(rho_msub.T, rho_msub.T)
# X_pca_man = pcam.fit_transform(gram)

X_pca = kpca.fit_transform(rho_msub.T)
X_pca_adv = kpca_adv.fit_transform(rho_msub.T)
# X_pca1 = kpca1.fit_transform(rho_msub.T)
X_pca2 = kpca2.fit_transform(rho_msub.T)
X_pca4 = kpca4.fit_transform(rho_msub.T)

X_back = kpca.inverse_transform(X_pca)
X_back_adv = kpca_adv.inverse_transform(X_pca_adv)
# X_back1 = kpca1.inverse_transform(X_pca1)
X_back2 = kpca2.inverse_transform(X_pca2)
X_back4 = kpca4.inverse_transform(X_pca4)


X_back_lin = pca.inverse_transform(X_pca_lin)
X_back_lin_fft = pca_fft.inverse_transform(X_pca_lin_fft)
X_back_lin_fft_inv = fftpack.ifftn(X_back_lin_fft)

# K = my_kernel(X_pca,X_pca)
# K.flat[:: nPoints + 1] += 1
# dual_coef_ = linalg.solve(K, rho_msub.T, sym_pos=True, overwrite_a=True)
# K = my_kernel(X_pca, X_pca)
# X_back_man_nonl = np.dot(K, dual_coef_).T

X_back_lin = X_back_lin.T
# X_back1 = X_back1.T
# X_back_man = X_back_man.T
X_back = X_back.T
X_back_adv = X_back_adv.T
# X_back1 = X_back1.T
X_back2 = X_back2.T
X_back4 = X_back4.T
X_back_lin_fft_inv = X_back_lin_fft_inv.T

error = rho_msub-X_back
error_norm = linalg.norm(error)
print('error_norm_rbf:', error_norm)  # Error in reconstruction using built-in rbf kpca
error_adv = rho_msub-X_back_adv
error_norm_adv = linalg.norm(error_adv)
print('error_norm_rbf with input alpha:', error_norm_adv)  # Error in reconstruction using built-in rbf kpca

# error1 = rho_msub-X_back1
# error_norm1 = linalg.norm(error1)
# print('error_norm_poly:', error_norm1)

error2 = rho_msub-X_back2
error_norm2 = linalg.norm(error2)
print('error_norm_sigmoid:', error_norm2)

error4 = rho_msub-X_back4
error_norm4 = linalg.norm(error4)
print('error_norm_laplacian:', error_norm4)

# error1 = rho_msub-X_back1
# error1_norm = linalg.norm(error1)
# print('error1_norm:', error1_norm)  # Error in reconstruction using built-in cosine kpca with
# a linear pre-image learning
# error_man = rho_msub-X_back_man
# error_norm_man = linalg.norm(error_man)
# print('error_norm_man:', error_norm_man)  # Error in reconstruction using precomputed cosine kpca with
# a linear pre-image learning
error_lin = rho_msub-X_back_lin
error_norm_lin = linalg.norm(error_lin)  # , ord=inf
print('error_norm_lin:', error_norm_lin)   # Error in reconstruction using built-in linear pca with
# error_lin_fft = fftout.real - X_back_lin_fft.T
error_lin_fft = rho_msub - X_back_lin_fft_inv
error_norm_lin_fft = linalg.norm(error_lin_fft)  # , ord=inf
print('error_norm_lin_fft:', error_norm_lin_fft)  # Error in reconstruction using built-in linear pca with
# error_nonl = rho_msub - X_back_man_nonl
# error_norm_nonl = linalg.norm(error_nonl)
# print('error_norm_nonl:', error_norm_nonl)  # Error in reconstruction using precomputed cosine kpca with
# a nonlinear pre-image learning
# exit()


# fftin_resh = fftout.real.reshape(2000, 20, 24, 4)
# fftout_resh = error_lin_fft.reshape(2000, 20, 24, 4)
error = rho_msub - X_back_lin_fft_inv
error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
             cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('fft_error.png')

error = rho_msub-X_back_lin
error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
             cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('ReconstructionError_pca.png')

error = rho_msub-X_back
error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
             cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('ReconstructionError_kpca.png')

error_adv = rho_msub-X_back_adv
error_adv = np.reshape(error_adv.T, newshape=(2000, 20, 24, 4))
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_adv[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
             cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('ReconstructionError_kpca_adv.png')

# error = rho_msub-X_back_man_nonl
# error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
# plt.figure()
# plt.rcParams.update({'font.size': 14})  # increase the font size
# mpl.rcParams['legend.fontsize'] = 15
# plt.xlabel("Longitude [deg]")
# plt.ylabel("Latitude [deg]")
# plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
#              cmap="inferno", levels=900)
# plt.colorbar()
# plt.grid(True)
# plt.tight_layout()
# plt.savefig('ReconstructionError_kpca_man.png')
plt.close('all')



error_norm_rbf: 12.724886233509306
error_norm_rbf with input alpha: 0.8572154456350575
error_norm_sigmoid: 48.416948290006495
error_norm_laplacian: 25.177185056304037
error_norm_lin: 1.2293381103564867
error_norm_lin_fft: 53.197648851987395
